In [1]:
%matplotlib inline
# %load_ext autoreload
# %autoreload 2

import sys; sys.path.insert(0, r'C:\Users\lukas\Dokumente\projects\esinet')
import pickle as pkl
import numpy as np
from copy import deepcopy
import mne
import seaborn as sns
import matplotlib.pyplot as plt
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info

plot_params = dict(surface='white', hemi='both', verbose=0)

In [2]:
info = get_info()
info['sfreq'] = 100
fwd = create_forward_model(info=info)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    2.2s remaining:    3.7s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    2.7s remaining:    1.6s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.9s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.2s finished


In [3]:
n_samples = 1000
duration_of_trial = 1.0
settings = dict(duration_of_trial=duration_of_trial, method='noise')

sim_lstm = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=n_samples)
# sim_lstm.save(f'simulations/sim_{n_samples}_{int(duration_of_trial*100)}points.pkl')
# sim_lstm_prime = Simulation(fwd, info, verbose=True, settings=settings_prime).simulate(n_samples=2000)

Simulate Source
doing the noise-based source simulation thing


  0%|          | 0/1000 [00:00<?, ?it/s]

Converting Source Data to mne.SourceEstimate object


  0%|          | 0/1000 [00:00<?, ?it/s]


Project sources to EEG...

Create EEG trials with noise...


  0%|          | 0/1000 [00:00<?, ?it/s]


Convert EEG matrices to a single instance of mne.Epochs...


In [5]:
import tensorflow as tf
epochs = 150
patience = 10
activation_function = 'relu'
loss = 'mean_squared_error'
dropout = 0.2
optimizer = tf.keras.optimizers.Adam(clipvalue=0.5) #  'adam'  # tf.keras.optimizers.RMSprop(learning_rate=0.0001)

# Dense net
model_params = dict(activation_function=activation_function, n_dense_layers=2, 
    n_dense_units=400, n_lstm_layers=0, model_type='v2')
train_params = dict(epochs=epochs, patience=patience, tensorboard=True, 
    dropout=dropout, loss=loss, optimizer=optimizer, return_history=True,
    batch_size=8)
# Train
# net_dense = Net(fwd, **model_params)
# _, history_dense = net_dense.fit(sim_lstm, **train_params)

# LSTM v2
model_params = dict(activation_function=activation_function, n_lstm_layers=2, 
    n_lstm_units=75, n_dense_layers=0, 
    model_type='v2')
train_params = dict(epochs=epochs, patience=patience, tensorboard=True, 
    dropout=0.1, loss=loss, optimizer=optimizer, return_history=True,
    batch_size=8)

# Train
net_lstm = Net(fwd, **model_params)
_, history_lstm = net_lstm.fit(sim_lstm, **train_params)

models = [net_dense, net_lstm]
model_names = ['Dense', 'LSTM']
# net_dense.save(r'models', name='dense-net-100points-noise')
# net_lstm.save(r'models', name='lstm-net-100points-noise')

Model: "LSTM_v2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
RNN_0 (Bidirectional)        (None, None, 150)         82200     
_________________________________________________________________
RNN_1 (Bidirectional)        (None, None, 150)         135600    
_________________________________________________________________
FC_Out (TimeDistributed)     (None, None, 1284)        193884    
Total params: 411,684
Trainable params: 411,684
Non-trainable params: 0
_________________________________________________________________
handle data
preprocess data
reshape data
fit model
Epoch 1/150
113/113 [==============================] - 20s 124ms/step - loss: 0.2310 - mae: 0.2659 - val_loss: 0.0922 - val_mae: 0.2434
Epoch 2/150
113/113 [==============================] - 12s 107ms/step - loss: 0.0887 - mae: 0.2377 - val_loss: 0.0851 - val_mae: 0.2324
Epoch 3/150
113/113 [==============================] - 12s 1